In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
# Create columns and target
columns = [
    "flightDate", "startingAirport", "destinationAirport", "travelDuration", "isBasicEconomy", "isRefundable", "isNonStop",
    "totalTravelDistance", "searchDaysBeforeFlight"
]

target = ["totalFare"]

In [5]:
# Load the data
file_path = Path("../Resources/flight_data_clean_all.csv")
df = pd.read_csv(file_path, skiprows=0)[:-2]

df.head()

,Unnamed: 0,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,totalTravelDistance,searchDaysBeforeFlight
0,0,2022-09-28,2022-10-05,SFO,ORD,0 days 07:05:00,0,0,0,451.60,1933.0,7
1,1,2022-09-28,2022-10-05,SFO,ORD,0 days 07:52:00,0,0,0,451.60,1933.0,7
2,2,2022-09-28,2022-10-05,SFO,ORD,0 days 08:34:00,0,0,0,451.60,1933.0,7
3,3,2022-09-28,2022-10-05,SFO,PHL,0 days 07:49:00,0,0,0,103.99,2590.0,7
4,4,2022-09-28,2022-10-05,SFO,PHL,0 days 13:19:00,0,0,0,109.59,2590.0,7


In [6]:
#drop first two columns
df = df.iloc[: , 2:]
df.head()

,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,totalTravelDistance,searchDaysBeforeFlight
0,2022-10-05,SFO,ORD,0 days 07:05:00,0,0,0,451.60,1933.0,7
1,2022-10-05,SFO,ORD,0 days 07:52:00,0,0,0,451.60,1933.0,7
2,2022-10-05,SFO,ORD,0 days 08:34:00,0,0,0,451.60,1933.0,7
3,2022-10-05,SFO,PHL,0 days 07:49:00,0,0,0,103.99,2590.0,7
4,2022-10-05,SFO,PHL,0 days 13:19:00,0,0,0,109.59,2590.0,7


In [7]:
#check for null values
df.isna().sum()

flightDate                     0
startingAirport                0
destinationAirport             0
travelDuration                 0
isBasicEconomy                 0
isRefundable                   0
isNonStop                      0
totalFare                      0
totalTravelDistance       116321
searchDaysBeforeFlight         0
dtype: int64

In [8]:
#drop null values
df = df.dropna()
df.isna().sum()

flightDate                0
startingAirport           0
destinationAirport        0
travelDuration            0
isBasicEconomy            0
isRefundable              0
isNonStop                 0
totalFare                 0
totalTravelDistance       0
searchDaysBeforeFlight    0
dtype: int64

In [9]:
# check df datatypes
df.dtypes

flightDate                 object
startingAirport            object
destinationAirport         object
travelDuration             object
isBasicEconomy              int64
isRefundable                int64
isNonStop                   int64
totalFare                 float64
totalTravelDistance       float64
searchDaysBeforeFlight      int64
dtype: object

In [10]:
# start new df and encode startingAirport and endingAirport
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df2 = df.copy()
df2['startingAirport'] = le.fit_transform(df2['startingAirport'])
df2['destinationAirport'] = le.fit_transform(df2['destinationAirport'])

In [11]:
#convert travelDuration to minutes
df2['travelDurationDays'] = pd.to_numeric(df2['travelDuration'].str.slice(0,1))
df2['travelDurationHours'] = pd.to_numeric(df2['travelDuration'].str.slice(7,9))
df2['travelDurationMinutes'] = pd.to_numeric(df2['travelDuration'].str.slice(10,12))
df2['travelDurationFinal'] = ((df2['travelDurationDays']*1440)+(df2['travelDurationHours']*60)+(df2['travelDurationMinutes']))
df2 = df2.drop(columns=['travelDurationDays','travelDurationHours','travelDurationMinutes','travelDuration'])
df2.head()

,flightDate,startingAirport,destinationAirport,isBasicEconomy,isRefundable,isNonStop,totalFare,totalTravelDistance,searchDaysBeforeFlight,travelDurationFinal
0,2022-10-05,15,13,0,0,0,451.60,1933.0,7,425
1,2022-10-05,15,13,0,0,0,451.60,1933.0,7,472
2,2022-10-05,15,13,0,0,0,451.60,1933.0,7,514
3,2022-10-05,15,14,0,0,0,103.99,2590.0,7,469
4,2022-10-05,15,14,0,0,0,109.59,2590.0,7,799


In [ ]:
# Create our features
X = df.copy()
X = X.drop("totalFare", axis=1)
X = pd.get_dummies(X, columns=["flightDate", "startingAirport", "destinationAirport", "travelDuration", "isBasicEconomy", "isRefundable", "isNonStop","totalTravelDistance", "searchDaysBeforeFlight"])


# Create our target
y = df["totalFare"]

In [ ]:
X.describe()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(x,y)

In [ ]:
y_pred = model.predict(x)

In [ ]:
plt.scatter(x,y)
plt.plot(x, y_pred, color="red")
plt.show()